# Sentiment Analysis usando Deep Learning para español en textos cortos

# 1. Data Cleaning 

We work on a dataset of tweets labelled as positive or negative, more information about the dataset 
can be found on http://www.sepln.org/workshops/tass/. 

# 2. Vocabulary

## 2.2 Getting train, validation and test subsets

In [ ]:
import lib.xmlreader as xml
import lib.utils as ut
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import gensim.models.word2vec

In [ ]:
train_docs= xml.readXML("../database/TASS/TASS2017/task1-Training.xml")
test_docs= xml.readXML("../database/TASS/TASS2017/task1-Development.xml")

In [ ]:
train_tweets = []
train_labels = []
for doc in train_docs:
    # train_tweets.append(ut.tokenize(doc.content, 0)['clean'])
    train_tweets.append(doc.content)
    train_labels.append(doc.polarity)

test_tweets = []
test_labels = []
for doc in test_docs:
    # test_tweets.append(ut.tokenize(doc.content, 0)['clean'])
    test_tweets.append(doc.content)
    test_labels.append(doc.polarity)

In [ ]:
len(train_tweets), len(test_tweets)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = (pd.Series(train_tweets), pd.Series(test_tweets), pd.Series(train_labels), pd.Series(test_labels))

In [113]:
fmt1_ = "Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive"
print(fmt1_.format(len(x_train),
      (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
      (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))

fmt3_ = "Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive"
print(fmt3_.format(len(x_test),
      (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
      (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))

Train set has total 736 entries with 56.79% negative, 43.21% positive
Test set has total 375 entries with 58.40% negative, 41.60% positive


# 3. Preparing Data

In [20]:
from gensim.models import KeyedVectors

def gensim_load_vec(path="../database/embeddings/SBW-vectors-300-min5.bin"):
    #use gensim_emb.wv.index2word if used this way to load vectors
    #gensim_emb = gensim.models.word2vec.Word2Vec.load(path)
    gensim_emb =  gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)
    vocab = gensim_emb.index2word
    vec = gensim_emb.syn0
    shape = gensim_emb.syn0.shape
    return gensim_emb, vec, shape, vocab

In [21]:
gensim_emb, vec, shape, vocab = gensim_load_vec()

/home/jose/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  
/home/jose/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':


In [114]:
#counter = CountVectorizer(strip_accents="unicode", tokenizer=ut.tokenizer)
counter = CountVectorizer(max_features=1240,tokenizer=ut.tokenizer)

In [115]:
X = counter.fit_transform(train_tweets + test_tweets)
print(X.shape)

(1111, 4227)


In [164]:
VOCAB_SIZE = X.shape[1]
embedding_matrix = np.zeros((VOCAB_SIZE, 300))
for word in list(counter.vocabulary_.keys()):
    try:
        i = counter.vocabulary_[word]
        embedding_matrix[i] = gensim_emb[word]
    except KeyError:
        pass
print(embedding_matrix.shape)

(4227, 300)


In [165]:
np.array_equal(embedding_matrix[counter.vocabulary_['hola']], gensim_emb['hola'])

True

In [166]:
sequences = []
for tweet in train_tweets+test_tweets:
    sentence = []
    for word in ut.tokenizer(tweet):
        try:
            i = counter.vocabulary_[word]
            sentence.append(i)
        except KeyError:
            pass
        
    sequences.append(sentence)

In [167]:
from keras.preprocessing.sequence import pad_sequences
x_train_seq = pad_sequences(sequences[:len(x_train)], maxlen=45)
x_test_seq  = pad_sequences(sequences[-len(x_test):], maxlen=45)
print('Shape of data train tensor:', x_train_seq.shape)
print('Shape of data val  tensor:', x_test_seq.shape)

Shape of data train tensor: (736, 45)
Shape of data val  tensor: (375, 45)


# Model: Convolutional Neural Network

In [168]:
import tensorflow as tf
import keras 

from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())


config = tf.ConfigProto(intra_op_parallelism_threads=4, \
                        inter_op_parallelism_threads=4, \
                        allow_soft_placement=True,\
                        device_count = {'CPU' : 1, 'GPU' : 0})

sess = tf.Session(config=config)
K.set_session(sess)
    
seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D

tweet_input = Input(shape=(45,), dtype='int32')

tweet_encoder = Embedding(VOCAB_SIZE, 300, weights=[embedding_matrix], input_length=45, trainable=True)(tweet_input)
bigram_branch = Conv1D(filters=20, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch  = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=20, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch  = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=20, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(128, activation='relu')(merged)
merged = Dropout(0.5)(merged)

merged = Dense(1)(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

['/job:localhost/replica:0/task:0/device:GPU:0']
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 45)           0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 45, 300)      1268100     input_20[0][0]                   
__________________________________________________________________________________________________
conv1d_53 (Conv1D)              (None, 44, 20)       12020       embedding_20[0][0]               
__________________________________________________________________________________________________
conv1d_54 (Conv1D)              (None, 43, 20)       18020       embedding_20[0][0]               
____________________________________________________________

In [169]:
from keras.callbacks import ModelCheckpoint

filepath="model/CNN_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(x_train_seq, y_train, batch_size=64, epochs=10,
                     validation_data=(x_test_seq, y_test), callbacks = [checkpoint])

Train on 736 samples, validate on 375 samples
Epoch 1/10
736/736 [==============================] - 2s 3ms/step - loss: 0.6938 - acc: 0.5598 - val_loss: 0.6760 - val_acc: 0.6400

Epoch 00001: val_acc improved from -inf to 0.64000, saving model to model/CNN_best_weights.01-0.6400.hdf5
Epoch 2/10
736/736 [==============================] - 1s 1ms/step - loss: 0.6381 - acc: 0.6359 - val_loss: 0.6465 - val_acc: 0.6160

Epoch 00002: val_acc did not improve from 0.64000
Epoch 3/10
736/736 [==============================] - 1s 1ms/step - loss: 0.5461 - acc: 0.7704 - val_loss: 0.6244 - val_acc: 0.6933

Epoch 00003: val_acc improved from 0.64000 to 0.69333, saving model to model/CNN_best_weights.03-0.6933.hdf5
Epoch 4/10
736/736 [==============================] - 1s 1ms/step - loss: 0.4379 - acc: 0.8750 - val_loss: 0.5801 - val_acc: 0.7173

Epoch 00004: val_acc improved from 0.69333 to 0.71733, saving model to model/CNN_best_weights.04-0.7173.hdf5
Epoch 5/10
736/736 [============================

In [170]:
#from keras.models import load_model

#loaded_CNN_model = load_model('model/CNN_best_weights.05-0.5400.hdf5')
#loaded_CNN_model.evaluate(x=x_val_seq, y=y_validation)
model.evaluate(x=x_test_seq, y=y_test)

375/375 [==============================] - 0s 176us/step


[0.5950926593144734, 0.7306666649182637]

In [123]:
model.metrics_names

['loss', 'acc']